<a href="https://colab.research.google.com/github/kishan5470/PredictiveAnalytics/blob/main/obtainin_the_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obtaining and cleaning the CSV file and dividing it into test and train set


In [ ]:
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,ConfusionMatrixDisplay
from sklearn.feature_extraction.text import TfidfVectorizer,ENGLISH_STOP_WORDS


In [ ]:
#Twitter API credentials
consumerKey='CnzAZRTO9eeuZbhTibGPCQaWU'
consumerSecret='Pvo0eySiKaKXWBYPaAEtIoyxhblaozpmejX6vWGo0gdYIkqSZX'
accessToken='1347704509162782720-WyEg5wqX5fmO5oezQB0bYCMluJisYg'
accessTokenSecret='tkJ3RfIMhb6frS1wii37PjPJeIXKsjGrlxV2QhCmMk7Jf'

In [ ]:
#Authentication object
authenticate=tweepy.OAuthHandler(consumerKey,consumerSecret)

#accesstoken
authenticate.set_access_token(accessToken,accessTokenSecret)

#api object
api=tweepy.API(authenticate,wait_on_rate_limit=True)

In [ ]:
# extracting 100 tweets from delhi police
posts=api.user_timeline(screen_name="DelhiPolice",count=200,lang="en",tweet_mode="extended")

In [ ]:
columns = ['Time', 'User', 'Tweet']
data = []
for tweet in posts:
    data.append([tweet.created_at, tweet.user.screen_name, tweet.full_text])

df = pd.DataFrame(data, columns=columns)
df.to_csv('tweets.csv')

In [ ]:
data=pd.read_csv('tweets.csv')
#creating function to cleam
def cleanTxt(text):
  text=re.sub(r'@[A-Za-z0-9]+','',text) #remove @mentions
  text=re.sub(r'#','',text) #remove #
  text=re.sub(r'RT[\s]+','',text) #remove retweets
  text=re.sub(r'https?:\/\/\S+','',text) #removes hyperlink
  text=re.sub(r'You may lodge your complaint on our website','',text)
  text=re.sub(r'complaint','',text)
  text=re.sub(r'Police Station','',text)
  text=re.sub(r'jurisdiction','',text)
  text=re.sub(r'may lodge','',text)
  text=re.sub(r'may','',text)
  text=re.sub(r'visit','',text)
  text=re.sub(r'available','',text)
  text=re.sub(r'DelhiPoliceUpdates','',text)
  text=re.sub(r'mayvisit','',text)
  text=re.sub(r'DelhiPoliceCares','',text)
  text=re.sub(r'reporting portal','',text)
  text=re.sub(r'crime reporting','',text)
  text=re.sub(r'helpline number','',text)
  text=re.sub(r'Please dial','',text)
  text=re.sub(r'Please Dial','',text)
  text=re.sub(r'Kindly Contact','',text)
  text=re.sub(r'nearest police','',text)
  text=re.sub(r'[^\u0000-\u05C0\u2100-\u214F]+','',text)

  return text

#cleaning the text
data['Tweet']=data['Tweet'].apply(cleanTxt)
data.rename({"Unnamed: 0":"local"}, axis="columns", inplace=True)
#data.drop(["a"], axis=1, inplace=True)

In [ ]:
data

,local,Time,User,Tweet
0,0,2022-10-15 21:02:02,DelhiPolice,You your on the cyber crime
1,1,2022-10-15 17:19:04,DelhiPolice,", \n \n\n 7 - \n\nAsiaCupWomen\n"
2,2,2022-10-15 17:07:07,DelhiPolice,112 for immediate assistance.
3,3,2022-10-15 16:01:23,DelhiPolice,' ' 75 8 9 7500 \n\n ...
4,4,2022-10-15 15:45:52,DelhiPolice,112 for immediate assistance.
...,...,...,...,...
195,195,2022-10-05 03:12:09,DelhiPolice,' ' \n\nVijayaDashmi\nDuss...
196,196,2022-10-05 01:58:52,DelhiPolice,_Guru_ You your on the cyber crime . Help ...
197,197,2022-10-05 01:56:52,DelhiPolice,_CIO: OperationCHAKRA \n\nCBI with State/UT po...
198,198,2022-10-04 20:15:19,DelhiPolice,_sayyed


In [ ]:
#Preprocessing the review text
#Specify the word pattern
pattern=r"[a-zA-Z]+"
vect = TfidfVectorizer(
    token_pattern=pattern,
    stop_words=ENGLISH_STOP_WORDS,
    ngram_range=(1,2),
    max_features=500
)
vect.fit(data["Tweet"])
tokenized_features=vect.transform(data["Tweet"])
features = pd.DataFrame(data=tokenized_features.toarray(),
                        columns=vect.get_feature_names_out())
features

,able,accused,action,agencies,amp,appropriate,appropriate action,area,arrested,ask,...,yadav bags,yadav cyber,year,year old,young,young monk,youth,youth jahangirpuri,youtuber,youtuber traced
0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
196,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
197,0.0,0.0,0.000000,0.0,0.163383,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
198,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
data["char_count"]=data["Tweet"].str.count(r"\S")
data["word_count"]=data["Tweet"].str.count(pattern)
data["avg_word_length"]=data["char_count"]/data["word_count"]


#np.isnan(data.values.any())
data[data==np.inf]=np.nan
data.fillna(data.mean(), inplace=True)
data.sample(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


,local,Time,User,Tweet,char_count,word_count,avg_word_length
174,174,2022-10-06 02:36:08,DelhiPolice,You contact the office of district D.C.P. wi...,56,15,3.733333
35,35,2022-10-13 09:46:31,DelhiPolice,You your on the cyber crime You also t...,39,11,3.545455
117,117,2022-10-08 05:58:56,DelhiPolice,", \n ' ' \n\n , , DelhiPoli...",22,1,22.000000


In [ ]:
# Define X and Y
X=pd.concat([features,data.loc[:,'char_count':]],axis=1)
y=data['Tweet']
X_train,X_test,y_train,y_test=train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=42
)
X_test.fillna(X_train.mean(), inplace=True)
X_test = X_test.fillna(X_train.mean())
#training a randomclassifier
rf=RandomForestClassifier(random_state=42).fit(X_train,y_train)

#predict the labels
y_pred=rf.predict(X_test)

#Print classification metrics
print(classification_report(y_test,y_pred))


                                                                                                                                                                                                                                                                                 precision    recall  f1-score   support

                                                                                                                                                                                                                                                                         
  or        1.00      1.00      1.00         1
                                                                                                                                                                                                                 
   ,       

 ,    ,           1090 ( )   

delhipolicecares        0.00      0.00      0.00         0
                                                            

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1